In [1]:
%load_ext autoreload
%autoreload 2
import pynwb
import os
import numpy as np

# DataJoint and DataJoint schema
import datajoint as dj

dj.config["filepath_checksum_size_limit"] = 1 * 1024**2

## We also import a bunch of tables so that we can call them easily
from spyglass.common import (
    ElectrodeGroup,
    Electrode,
    FirFilterParameters,
    IntervalList,
    Session,
    Task,
    TaskEpoch,
    Nwbfile,
    AnalysisNwbfile,
    NwbfileKachery,
    AnalysisNwbfileKachery,
    interval_list_contains,
    interval_list_contains_ind,
    interval_list_excludes,
    interval_list_excludes_ind,
    interval_list_intersect,
    get_electrode_indices,
)

from spyglass.lfp.v1 import (
    LFPElectrodeGroup,
    LFPSelection,
    LFPV1,
)
from spyglass.lfp.lfp_merge import LFPOutput


from spyglass.lfp.analysis.v1.lfp_band import LFPBandSelection,LFPBandV1,FirFilterParameters
# from spyglass.lfp_band.lfp_band_merge import LFPBandOutput
import os
import warnings
from tqdm import tqdm
warnings.simplefilter("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore", category=ResourceWarning)
warnings.simplefilter("ignore", category=UserWarning)

[2023-12-06 07:59:41,316][INFO]: Connecting sambray@lmf-db.cin.ucsf.edu:3306
[2023-12-06 07:59:41,418][INFO]: Connected sambray@lmf-db.cin.ucsf.edu:3306


In [6]:
FirFilterParameters().fetch('filter_name')

filter_names = ['Slow Gamma 25-55 Hz','Fast Gamma 65-100 Hz']

filter_name = filter_names[1]

FirFilterParameters & {'filter_name':filter_name} 

filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter,filter_type,filter_low_stop lowest frequency for stop band for low frequency side of filter,filter_low_pass lowest frequency for pass band of low frequency side of filter,filter_high_pass highest frequency for pass band for high frequency side of filter,filter_high_stop highest frequency for stop band of high frequency side of filter,filter_comments comments about the filter,filter_band_edges numpy array containing the filter bands (redundant with individual parameters),filter_coeff numpy array containing the filter coefficients
Fast Gamma 65-100 Hz,1000,bandpass,55.0,65.0,100.0,110.0,slow gamma filter for 1 Khz data,=BLOB=,=BLOB=


## Define Animal Sessions

In [27]:
animal_names = ["Banner","Danvers","Odins","Totoro","Wallie","Winnie","Yoshi","Olive", "Frodo", "Bilbo"]
from spyglass.common import Session

nwb_file_names = []
interval_list_names = []
for animal in animal_names:
    dataset = LFPV1 * Session & {'subject_id':animal}
    nwb_file_names.extend(dataset.fetch('nwb_file_name'))    
    interval_list_names.extend(dataset.fetch('target_interval_list_name'))


In [21]:
for nwb_file_name in set(nwb_file_names):
    for filter_name in filter_names:
        break
        continue
    # break
    print(filter_name)
    print(LFPBandSelection() &  {'nwb_file_name':nwb_file_name})

Slow Gamma 25-55 Hz


lfp_merge_id,filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter,nwb_file_name name of the NWB file,target_interval_list_name descriptive name of this interval list,lfp_band_sampling_rate the sampling rate for this band,min_interval_len the minimum length of a valid interval to filter
e5c5b785-c54a-4a2b-9b2f-6e6e921d17fe,Theta 5-11 Hz,1000,Banner20220224_.nwb,pos 0 valid times,100,1.0
1312bf55-0524-fda0-fce8-1eb34267a4b1,Theta 5-11 Hz,1000,Banner20220224_.nwb,pos 1 valid times,100,1.0
f0079a54-0dfd-0260-def3-6288cabdd2f8,Theta 5-11 Hz,1000,Banner20220224_.nwb,pos 2 valid times,100,1.0
e90fb226-87d8-c1a3-be22-cfaa18277075,Theta 5-11 Hz,1000,Banner20220224_.nwb,pos 3 valid times,100,1.0
aed3cb7e-5ced-5156-50a6-e81924fa5ea1,Theta 5-11 Hz,1000,Banner20220224_.nwb,pos 4 valid times,100,1.0
70185e7b-9db7-cb04-22f5-c147ed1baa6c,Theta 5-11 Hz,1000,Banner20220224_.nwb,pos 5 valid times,100,1.0
70185e7b-9db7-cb04-22f5-c147ed1baa6c,Theta 5-11 Hz,1000,Banner20220224_.nwb,pos 5 valid times,1000,1.0
d626748f-7e9d-23c8-09d9-efc54f4059fa,Theta 5-11 Hz,1000,Banner20220224_.nwb,pos 6 valid times,100,1.0
6e903435-ae63-f8fe-7e5f-c37e6dadb8c8,Theta 5-11 Hz,1000,Banner20220224_.nwb,pos 7 valid times,100,1.0


In [26]:
LFPV1()

nwb_file_name name of the NWB file,lfp_electrode_group_name the name of this group of electrodes,target_interval_list_name descriptive name of this interval list,filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter,analysis_file_name name of the file,interval_list_name descriptive name of this interval list,lfp_object_id the NWB object ID for loading this object from the file,"lfp_sampling_rate the sampling rate, in HZ"
arthur20220315_.nwb,all_tets_arthur,r1_r2,LFP 0-400 Hz,30000,arthur20220315_81AZEP1XDH.nwb,lfp_all_tets_arthur_r1_r2_valid times,e58117d5-41a7-408c-b4e5-c43167f083d7,1000.0
arthur20220315_.nwb,all_tets_arthur,r2_r3,LFP 0-400 Hz,30000,arthur20220315_8PB45D84KO.nwb,lfp_all_tets_arthur_r2_r3_valid times,ad302cd5-a998-4f1b-80b6-edb9fdea7999,1000.0
arthur20220316_.nwb,all_tets_arthur,r1_r2,LFP 0-400 Hz,30000,arthur20220316_J1M0E160MI.nwb,lfp_all_tets_arthur_r1_r2_valid times,99d2016e-c18a-4b85-9ccb-e832f44c783c,1000.0
arthur20220316_.nwb,all_tets_arthur,r2_r3,LFP 0-400 Hz,30000,arthur20220316_I9QVY08308.nwb,lfp_all_tets_arthur_r2_r3_valid times,ca53e85b-e9f0-42b2-be6f-31565bad28c4,1000.0
arthur20220317_.nwb,all_tets_arthur,r1_r2,LFP 0-400 Hz,30000,arthur20220317_X0R19HUN0X.nwb,lfp_all_tets_arthur_r1_r2_valid times,3f904bd7-e8b5-4b2e-9a36-6cf3ee628141,1000.0
arthur20220317_.nwb,all_tets_arthur,r2_r3,LFP 0-400 Hz,30000,arthur20220317_ZF77OL0D5C.nwb,lfp_all_tets_arthur_r2_r3_valid times,9100fb9b-7906-4852-b252-4d2a3bacbfc4,1000.0
arthur20220318_.nwb,all_tets_arthur,r1_r2,LFP 0-400 Hz,30000,arthur20220318_YZNI7VFTJV.nwb,lfp_all_tets_arthur_r1_r2_valid times,06f24d41-437b-440a-9c95-34494dabdb6d,1000.0
arthur20220318_.nwb,all_tets_arthur,r2_r3,LFP 0-400 Hz,30000,arthur20220318_1LZMUQ09VB.nwb,lfp_all_tets_arthur_r2_r3_valid times,1e0d14e9-25f2-470e-b7f1-ea524b64c86a,1000.0
arthur20220319_.nwb,all_tets_arthur,r2_r2,LFP 0-400 Hz,30000,arthur20220319_BOE1MGTXW6.nwb,lfp_all_tets_arthur_r2_r2_valid times,312e6bce-eb5a-40fc-b388-4f59e6238067,1000.0
arthur20220319_.nwb,all_tets_arthur,r2_r3,LFP 0-400 Hz,30000,arthur20220319_HJ2E2Y4MSS.nwb,lfp_all_tets_arthur_r2_r3_valid times,1a0a21c9-e5a5-47d5-9cbb-d77d1c55119e,1000.0


## Populate

In [32]:
# desired sampling rate
lfp_band_sampling_rate = 1000
missing_lfp = []


for nwb_file_name, interval_list_name in zip(nwb_file_names,interval_list_names):
    
    for filter_name in filter_names:
                
        basic_key = {"nwb_file_name": nwb_file_name,'target_interval_list_name':interval_list_name}
        
        #check that lfp exists for this animal
        if len(LFPV1() & basic_key)==0:
            missing_lfp.append(basic_key)
            continue
        if len(LFPV1 & basic_key)>1:
            electrode_group_names = (LFPV1() & basic_key).fetch("lfp_electrode_group_name")
            electrode_group_name =[x for x in electrode_group_names if nwb_file_name.split('202')[0] in x][0]
            basic_key["lfp_electrode_group_name"] = electrode_group_name
        if not (LFPV1 & basic_key).fetch1('lfp_sampling_rate')==1000:
            continue
          
        # skip if already in the selection table
        if LFPBandSelection() & {'nwb_file_name':nwb_file_name,'target_interval_list_name':interval_list_name,
                                 'filter_name':filter_name,'lfp_band_sampling_rate':lfp_band_sampling_rate}:
            continue    
        #get lfp's sampling rate
        lfp_sampling_rate = (LFPV1() & basic_key ).fetch1(
            "lfp_sampling_rate"
        )
        
        #get lfp filter sampling_rate
        lfp_filter_sampling_rate = (LFPV1() & basic_key).fetch1(
            "filter_sampling_rate"
        )
        
        #get lfp filter name
        lfp_filter_name = (LFPV1() & basic_key).fetch1(
            "filter_name"
        )
        
        # make lfp selection key
        lfp_s_key = basic_key.copy()
        lfp_s_key["target_interval_list_name"] = interval_list_name
        lfp_s_key["filter_name"] = lfp_filter_name
        lfp_s_key["filter_sampling_rate"] = lfp_filter_sampling_rate
        
        # get the electrodes the lfp was run on
        lfp_electrode_group_name = (LFPV1 & lfp_s_key).fetch1('lfp_electrode_group_name')
        lfp_eg_key = {
                "nwb_file_name": nwb_file_name,
                "lfp_electrode_group_name": lfp_electrode_group_name,
                }
        electrode_list = list(np.asarray((LFPV1 & basic_key).fetch_nwb()[0]['lfp'].electrodes.to_dataframe().index))
        
        # remake lfp selection key
        lfp_s_key = basic_key.copy()
        lfp_s_key["target_interval_list_name"] = interval_list_name
        lfp_s_key["filter_name"] = lfp_filter_name
        lfp_s_key["filter_sampling_rate"] = lfp_filter_sampling_rate
        
        # set the reference to -1 to indicate no reference for all channels
        ref_elect = [-1]
        
        # we also need the uuid for the LFP object
        lfp_merge_id = (LFPOutput.LFPV1 & lfp_s_key).fetch1('merge_id')
        
        print(filter_name, nwb_file_name, interval_list_name, lfp_merge_id)
        # #make Band selection table entry
        LFPBandSelection().set_lfp_band_electrodes(
                nwb_file_name=nwb_file_name,
                lfp_merge_id=lfp_merge_id,
                electrode_list=electrode_list,
                filter_name=filter_name,
                interval_list_name=interval_list_name,
                reference_electrode_list=ref_elect,
                lfp_band_sampling_rate=lfp_band_sampling_rate,
            )
        
        # lfp_band_key = (
        #     LFPBandSelection
        #     & {
        #         "lfp_merge_id": lfp_merge_id,
        #         "filter_name": filter_name,
        #         "lfp_band_sampling_rate": lfp_band_sampling_rate,
        #         "target_interval_list_name":interval_list_name,
        #     }
        # ).fetch1("KEY")
        # if LFPBandV1() & lfp_band_key:
        #     continue #already populated
        # # lfp_band_key['lfp_electrode_group_name']="full_probe"
        # LFPBandV1().populate(LFPBandSelection()&lfp_band_key)


[2023-12-06 08:55:02,487][WARNING]: Skipped checksum for file with hash: 775580b9-db04-b8d9-0cc5-eb5348656135, and path: /stelmo/nwb/analysis/Wallie20220922/Wallie20220922_4DOLJYTJS7.nwb


Fast Gamma 65-100 Hz Wallie20220922_.nwb pos 7 valid times c86fe31e-493b-90a8-4da9-47687d0e91e7


[2023-12-06 08:55:03,583][WARNING]: Skipped checksum for file with hash: 61750ec6-3a2f-6c1c-2af2-9cb216dd1b26, and path: /stelmo/nwb/analysis/Wallie20220922/Wallie20220922_FWPAVOCR1X.nwb


Slow Gamma 25-55 Hz Wallie20220922_.nwb pos 8 valid times c41fa96f-6c66-5f66-94cc-a9f8a8530c57


[2023-12-06 08:55:05,161][WARNING]: Skipped checksum for file with hash: 61750ec6-3a2f-6c1c-2af2-9cb216dd1b26, and path: /stelmo/nwb/analysis/Wallie20220922/Wallie20220922_FWPAVOCR1X.nwb


...

Fast Gamma 65-100 Hz Bilbo20230804_.nwb pos 3 valid times 58939102-7774-fe1e-7361-726242519b50


[2023-12-06 09:18:26,161][WARNING]: Skipped checksum for file with hash: 3a31e642-721c-d401-d697-5019361f3ab5, and path: /stelmo/nwb/analysis/Bilbo20230804/Bilbo20230804_3QTOB4AKX6.nwb


Slow Gamma 25-55 Hz Bilbo20230804_.nwb pos 4 valid times d1512c83-6945-f029-72bb-2453a666a918


[2023-12-06 09:18:28,526][WARNING]: Skipped checksum for file with hash: 3a31e642-721c-d401-d697-5019361f3ab5, and path: /stelmo/nwb/analysis/Bilbo20230804/Bilbo20230804_3QTOB4AKX6.nwb


Fast Gamma 65-100 Hz Bilbo20230804_.nwb pos 4 valid times d1512c83-6945-f029-72bb-2453a666a918


In [30]:
LFPBandSelection() & {'filter_name':'Slow Gamma 25-55 Hz'}

[2023-12-06 08:51:36,901][WARNING]: MySQL server has gone away. Reconnecting to the server.


lfp_merge_id,filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter,nwb_file_name name of the NWB file,target_interval_list_name descriptive name of this interval list,lfp_band_sampling_rate the sampling rate for this band,min_interval_len the minimum length of a valid interval to filter
00aedd21-9c6e-39e1-4e91-04a00f5dc57c,Slow Gamma 25-55 Hz,1000,Wallie20220922_.nwb,pos 5 valid times,1000,1.0
01db9906-95a3-afa2-adf2-8307a6b66905,Slow Gamma 25-55 Hz,1000,Totoro20220531_.nwb,pos 1 valid times,1000,1.0
0350db5a-d32b-8745-b3b0-10dfab3fdb4b,Slow Gamma 25-55 Hz,1000,Odins20220216_.nwb,pos 7 valid times,1000,1.0
0378ca90-bd31-def0-aca8-b3a705adaf76,Slow Gamma 25-55 Hz,1000,Odins20220110_.nwb,pos 2 valid times,1000,1.0
046ceb75-8820-b22e-21e4-a7c3d942b99b,Slow Gamma 25-55 Hz,1000,Totoro20220527_.nwb,pos 1 valid times,1000,1.0
04d0d8c3-db8c-528e-a03f-131749f785b3,Slow Gamma 25-55 Hz,1000,Totoro20220610_.nwb,pos 5 valid times,1000,1.0
0601305a-fadf-eea1-d54f-694abfec91fa,Slow Gamma 25-55 Hz,1000,Banner20220225_.nwb,pos 2 valid times,1000,1.0
07a627ed-018c-78bb-08d2-f02357811af8,Slow Gamma 25-55 Hz,1000,Wallie20220913_.nwb,pos 5 valid times,1000,1.0
08cb60ed-cb5a-09ea-9420-551854ee3f88,Slow Gamma 25-55 Hz,1000,Totoro20220530_.nwb,pos 1 valid times,1000,1.0
093796f3-db61-7b08-d0d3-7841c2d2f9c9,Slow Gamma 25-55 Hz,1000,Odins20220216_.nwb,pos 5 valid times,1000,1.0


In [43]:
filter_name = filter_names[0]
(LFPBandSelection() & {'filter_name':filter_name}).proj()
LFPBandV1().populate((LFPBandSelection() & {'filter_name':filter_name}).proj() ,display_progress=True,processes=32)


Processes:   0%|          | 0/674 [00:00<?, ?it/s][2023-12-06 10:44:50,325][WARNING]: Skipped checksum for file with hash: 44704709-7097-5086-3812-4f0c30c8e691, and path: /stelmo/nwb/analysis/Wallie20220911/Wallie20220911_Y5H7RNRESZ.nwb
[2023-12-06 10:44:50,335][WARNING]: Skipped checksum for file with hash: 25d8b8cd-4a4e-b4ba-2252-776d28bb4718, and path: /stelmo/nwb/analysis/Winnie20220713/Winnie20220713_YYUM4C0GTJ.nwb
[2023-12-06 10:44:50,337][WARNING]: Skipped checksum for file with hash: 09dc5a4b-74f2-7763-9da8-41ef3138b720, and path: /stelmo/nwb/analysis/Totoro20220606/Totoro20220606_Y2D8H6PYY4.nwb
[2023-12-06 10:44:50,342][WARNING]: Skipped checksum for file with hash: 1dbff963-9ba7-3014-3a43-fec29d480df2, and path: /stelmo/nwb/analysis/Frodo20230809/Frodo20230809_239VGF5BXR.nwb
[2023-12-06 10:44:50,346][WARNING]: Skipped checksum for file with hash: 0500b1b8-30d7-d1b0-fc3f-e69379ef3c7c, and path: /stelmo/nwb/analysis/Winnie20220718/Winnie20220718_JN19HSGFES.nwb
[2023-12-06 10:44

Writing new NWB file Olive20220630_CAFXJIRTYT.nwb
Writing new NWB file Olive20220629_A3GR8RTPJI.nwbWriting new NWB file Banner20220126_5SDTBVF9P2.nwb

Writing new NWB file Odins20220107_Y3L4TBLY06.nwb
Writing new NWB file Olive20220629_HGET3ODR90.nwb
Writing new NWB file Olive20220629_S153MTOXWE.nwb
Writing new NWB file Olive20220707_8FCXAJMMYI.nwb
Writing new NWB file Yoshi20220516_65GK83SXCO.nwb
Writing new NWB file Yoshi20220516_9KJDG1GNZN.nwb
Writing new NWB file Olive20220701_8ZEF9EER3K.nwb
Writing new NWB file Olive20220706_C7LCMOTKV1.nwb
Writing new NWB file Winnie20220713_0M2DS005DV.nwb
Writing new NWB file Wallie20220911_I1HHQ4WSWJ.nwb
Writing new NWB file Totoro20220616_M5JJ8ABMEM.nwb
Writing new NWB file Totoro20220617_HMS52R7DIC.nwbWriting new NWB file Totoro20220617_NPLRQBQKZ7.nwb

Writing new NWB file Winnie20220713_FPA7K2953U.nwb
Writing new NWB file Olive20220708_KN7KXMUACD.nwb
Writing new NWB file Winnie20220718_EVEI6BDQQY.nwb
Writing new NWB file Winnie20220718_X8AATA

In [37]:
%debug

> /home/sambray/mambaforge-pypy3/envs/spyglass/lib/python3.9/site-packages/ghostipy/dsp/convolution.py(31)osconvolve()
     29                              " input_index_bounds but must have 2 elements")
     30         if not input_index_bounds[0] < input_index_bounds[1]:
---> 31             raise ValueError(f"'input_index_bounds' {input_index_bounds} is not"
     32                              f" strictly increasing")
     33         try:

> /home/sambray/mambaforge-pypy3/envs/spyglass/lib/python3.9/site-packages/ghostipy/dsp/filtering.py(84)filter_data_fir()
     82     """
     83 
---> 84     return osconvolve(
     85         data,
     86         b,

> /home/sambray/Documents/spyglass/src/spyglass/common/common_filter.py(467)filter_data()
    465             indices.append((frm, to))
    466 
--> 467             shape, _ = gsp.filter_data_fir(
    468                 data,
    469                 filter_coeff,

> /home/sambray/Documents/spyglass/src/spyglass/lfp/analysis/v1/lfp

In [46]:
basic_key
period_ms
(LFPV1 & basic_key)#.fetch1('lfp_sampling_rate')
LFPV1() & {'nwb_file_name':'Winnie20220713_.nwb'}
animal = Session & {'nwb_file_name':nwb_file_name}.fetch1('subject_id')
electrode_group_name = f'tetrode_sample_{animal}'


nwb_file_name name of the NWB file,lfp_electrode_group_name the name of this group of electrodes,target_interval_list_name descriptive name of this interval list,filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter,analysis_file_name name of the file,interval_list_name descriptive name of this interval list,lfp_object_id the NWB object ID for loading this object from the file,"lfp_sampling_rate the sampling rate, in HZ"
Winnie20220713_.nwb,tetrode_sample_Winnie,pos 0 valid times,LFP 0-400 Hz,30000,Winnie20220713_C52XDICU6D.nwb,lfp_tetrode_sample_Winnie_pos 0 valid times_valid times,a89c590f-290b-4f9c-a568-b9ae67eee96d,1000.0
Winnie20220713_.nwb,tetrode_sample_Winnie,pos 1 valid times,LFP 0-400 Hz,30000,Winnie20220713_FURB0FGK1A.nwb,lfp_tetrode_sample_Winnie_pos 1 valid times_valid times,2d99415e-6646-44e1-8745-d0af2912f9b1,1000.0
Winnie20220713_.nwb,tetrode_sample_Winnie,pos 10 valid times,LFP 0-400 Hz,30000,Winnie20220713_9GNIXRYWTS.nwb,lfp_tetrode_sample_Winnie_pos 10 valid times_valid times,48bc053f-ff5b-460f-838b-cefd8eba4b0d,1000.0
Winnie20220713_.nwb,tetrode_sample_Winnie,pos 11 valid times,LFP 0-400 Hz,30000,Winnie20220713_824JXQAB78.nwb,lfp_tetrode_sample_Winnie_pos 11 valid times_valid times,87e1b640-4938-4974-bce2-766f61c2e66f,1000.0
Winnie20220713_.nwb,tetrode_sample_Winnie,pos 12 valid times,LFP 0-400 Hz,30000,Winnie20220713_FLOILA8D0V.nwb,lfp_tetrode_sample_Winnie_pos 12 valid times_valid times,21aa2368-261b-4a1e-9e3b-151727e7a8b9,1000.0
Winnie20220713_.nwb,tetrode_sample_Winnie,pos 13 valid times,LFP 0-400 Hz,30000,Winnie20220713_YYUM4C0GTJ.nwb,lfp_tetrode_sample_Winnie_pos 13 valid times_valid times,599008ed-d7b4-4aca-9a65-b7cb6619a026,1000.0
Winnie20220713_.nwb,tetrode_sample_Winnie,pos 14 valid times,LFP 0-400 Hz,30000,Winnie20220713_49PBQ0QLK5.nwb,lfp_tetrode_sample_Winnie_pos 14 valid times_valid times,5aac489d-2f02-49eb-92e3-46d31390c193,1000.0
Winnie20220713_.nwb,tetrode_sample_Winnie,pos 2 valid times,LFP 0-400 Hz,30000,Winnie20220713_HBKDTJIJAR.nwb,lfp_tetrode_sample_Winnie_pos 2 valid times_valid times,3e8e3eaf-a12d-4b0f-b218-6e2deac10f30,1000.0
Winnie20220713_.nwb,tetrode_sample_Winnie,pos 3 valid times,LFP 0-400 Hz,30000,Winnie20220713_I2WK0NS3K8.nwb,lfp_tetrode_sample_Winnie_pos 3 valid times_valid times,e7e39ca8-98e9-4dc2-bded-2a0b2980ff07,1000.0
Winnie20220713_.nwb,tetrode_sample_Winnie,pos 4 valid times,LFP 0-400 Hz,30000,Winnie20220713_W4TKS379ID.nwb,lfp_tetrode_sample_Winnie_pos 4 valid times_valid times,bf864bf0-6b1a-4043-a34a-65f036dd3e27,1000.0


In [10]:
lfp_band_key

(LFPV1 & basic_key).fetch_nwb()[0]['lfp']

[2023-10-20 08:18:10,755][WARNING]: Skipped checksum for file with hash: 1f41ed1f-df73-533d-6c95-5cb459607516, and path: /stelmo/nwb/analysis/Bilbo20230725/Bilbo20230725_JRH9I6411O.nwb


filtered data pynwb.ecephys.ElectricalSeries at 0x140401433184672
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (2016507, 14), type "<i2">
  description: filtered data
  electrodes: electrodes <class 'hdmf.common.table.DynamicTableRegion'>
  interval: 1
  offset: 0.0
  resolution: -1.0
  timestamps: <HDF5 dataset "timestamps": shape (2016507,), type "<f8">
  timestamps_unit: seconds
  unit: volts

## Check missing LFP's

In [13]:
print(missing_lfp)

'Banner20220225_.nwb'